In [ ]:
! pip install soundfile
! pip install SpeechRecognition
! pip install language-tool-python
# Run this first before every session

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 142 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Libraries Import 

import soundfile as sf
import speech_recognition as sr
import requests
import re
import nltk
import language_tool_python

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# Reading the downloaded audio file and finding the length
file = 'ns002.wav'
f =  sf.SoundFile(file)
audio_length = int(len(f) / f.samplerate) + (len(f)%f.samplerate > 0)    #round up to make sure we get the entire file
print(f"Length of audio: {audio_length} seconds") # Lddength of the audio file in seconds

Length of audio: 78 seconds


In [ ]:
# Number of loops to make to recognize audio file
loop_number = int(audio_length/10) + (audio_length% 10 >0) #calculate the number of required loops


#create a function that loops through the audio file and transcribes
def transcribe(loops, starting_increment):
    audio_text1 = [] 
    increment = starting_increment 
    for i in range(loops):
        r = sr.Recognizer()
        with sr.AudioFile(file) as source:
            audio = r.record(source, offset = (increment) , duration= 10)
        string = r.recognize_google(audio)
        audio_text1.append(string)
        increment += 10
    return audio_text1

In [ ]:
audio_text = transcribe(loop_number, 0)


In [ ]:
audio_text

["it was time for a haircut didn't even have to look in the mirror even though he's going bald he knew that he needed to cut his hair ever",
 'two weeks he had a ton of hair on the top of his head his hair was at the crown he still have',
 'eisenback it was what they call Salt and Pepper a mixture of gray hair and dark brown hair it was only a few years to figure',
 'until the salt and pepper became just salt he never let his hair grow for more than two weeks the longer it got the worse it looked',
 'thought his brother newspaper over the bathroom sink so that no hair went down the drain he plugged in the Clippers and started cutting his hair',
 'he started at the back of his head went to the sides and finished on the top every minute or so he had to clean the hair out of the blades',
 'with an old toothbrush he picked up a mirror to check out the back of his head everything looked okay he carried the news',
 'back out to the kitchen and shook the hair clippings into the trash can then

In [ ]:

full_text = "".join(audio_text)
full_text

"it was time for a haircut didn't even have to look in the mirror even though he's going bald he knew that he needed to cut his hair evertwo weeks he had a ton of hair on the top of his head his hair was at the crown he still haveeisenback it was what they call Salt and Pepper a mixture of gray hair and dark brown hair it was only a few years to figureuntil the salt and pepper became just salt he never let his hair grow for more than two weeks the longer it got the worse it lookedthought his brother newspaper over the bathroom sink so that no hair went down the drain he plugged in the Clippers and started cutting his hairhe started at the back of his head went to the sides and finished on the top every minute or so he had to clean the hair out of the bladeswith an old toothbrush he picked up a mirror to check out the back of his head everything looked okay he carried the newsback out to the kitchen and shook the hair clippings into the trash can then he took a shower"

In [ ]:
# Temporary punctuator to trial error handling

url_punctuator = 'http://bark.phon.ioc.ee/punctuator'
r = requests.post(url_punctuator, data = {'text': full_text})
punc_text = r.text
punc_text

"It was time for a haircut didn't even have to look in the mirror. Even though he's going bald, he knew that he needed to cut his hair evertwo weeks. He had a ton of hair on the top of his head. His hair was at the crown he still haveeisenback. It was what they call: Salt and Pepper, a mixture of gray, hair and dark brown hair. It was only a few years to figureuntil the salt and pepper became just salt. He never let his hair grow for more than two weeks. The longer it got, the worse it lookedthought his brother newspaper over the bathroom sink so that no hair went down the drain he plugged in the Clippers and started cutting his hairhe started at the back of his head, went to the sides and finished on the top. Every minute, or so he had to clean the hair out of the bladeswith, an old toothbrush. He picked up a mirror to check out the back of his head. Everything looked okay, he carried the newsback out to the kitchen and shook the hair clippings into the trash. Can then he took a showe

In [ ]:
# Splitting text by sentences
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

sentence_list = split_into_sentences(punc_text)
sentence_list

["It was time for a haircut didn't even have to look in the mirror.",
 "Even though he's going bald, he knew that he needed to cut his hair evertwo weeks.",
 'He had a ton of hair on the top of his head.',
 'His hair was at the crown he still haveeisenback.',
 'It was what they call: Salt and Pepper, a mixture of gray, hair and dark brown hair.',
 'It was only a few years to figureuntil the salt and pepper became just salt.',
 'He never let his hair grow for more than two weeks.',
 'The longer it got, the worse it lookedthought his brother newspaper over the bathroom sink so that no hair went down the drain he plugged in the Clippers and started cutting his hairhe started at the back of his head, went to the sides and finished on the top.',
 'Every minute, or so he had to clean the hair out of the bladeswith, an old toothbrush.',
 'He picked up a mirror to check out the back of his head.',
 'Everything looked okay, he carried the newsback out to the kitchen and shook the hair clippings

In [ ]:
# Checking for grammatical errors using parsers via the language-tool-python library

tool = language_tool_python.LanguageTool('en-US')

for i in range(len(sentence_list)):
  matches = tool.check(sentence_list[i])
  if len(matches) > 0:
    print(f"Grammatical error in sentence {i+1}. Recommended fix: '{tool.correct(sentence_list[i])}'") # Not perfect. There are some cases that the parser can't correct

Unzipping /tmp/tmp3_tex1uy.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


Grammatical error in sentence 2. Recommended fix: 'Even though he's going bald, he knew that he needed to cut his hair ever two weeks.'
Grammatical error in sentence 4. Recommended fix: 'His hair was at the crown he still haveeisenback.'
Grammatical error in sentence 6. Recommended fix: 'It was only a few years to figure until the salt and pepper became just salt.'
Grammatical error in sentence 8. Recommended fix: 'The longer it got, the worse it looked thought his brother newspaper over the bathroom sink so that no hair went down the drain he plugged in the Clippers and started cutting his hearse started at the back of his head, went to the sides and finished on the top.'
Grammatical error in sentence 9. Recommended fix: 'Every minute, or so he had to clean the hair out of the blades with, an old toothbrush.'
Grammatical error in sentence 11. Recommended fix: 'Everything looked okay, he carried the news back out to the kitchen and shook the hair clippings into the trash.'


In [ ]:
text_file = open("result.txt","a")
for i in sentence_list:
  text_file.write(f"{i}\n")
text_file.close()